## Define all Requirements
- `credits_required`
- Boolean value of requirements
  - `matsci`
  - `bio`
  - `probstat`
  - `design_depth`
- course_number of requirements
  - `major_requirements`
  - `bio_requirements` 
  - `design_depth_requirements` 
  - `probstat_requirements` 
  - `matsci_requirements` 

In [9]:
credits_required = {
    "AHS": 28,
    "ENG": 46,
    "MTH/SCI": 30,
    "MTH": 10,
    'SCI':0,
    "OFYI": 1,
    "TOTAL": 120,
}
matsci = False
bio = False
probstat = False
design_depth = False

In [10]:
major_requirements = {
    "E:C": {
        "ENGR2510": [True, 0],
        "ENGR3525": [False, 1],
        "ENGR3599A/ENGR3520": [False, 1],
        "ENGR3220/ENGR3410/ENGR3540": [False, 1],
        "ENGR3590/ENGR3599": [False, 1],
    },
    'ECE':{"ENGX2010":[False,0],"ENGR2510":[True, 0], "ENGX2134":[False,1],"ENGR2410":[False,1],"ENGR2420":[False,1], 
                             "ENGR2199B":[False,1],"ENGR3410":[False,1],"ENGR3415":[False,2],"ENGR3420":[False,2],"ENGR3110":[False,2],
                             "ENGR3370":[False,2],"ENGR3390":[False,1],"ENGR3392":[False,2],"ENGR3415":[False,2],"ENGR3420":[False,2],
                             "ENGR3426":[False,2],"ENGR3430":[False,3],"ENGR3440":[False,3],"ENGR3499":[False,2],"MTH2110":[False,0]
                             },
    "MechE": {
        "ENGX2010": [False, 0],
        "ENGR2320": [False, 0],
        "ENGX2134": [False, 0],
        "ENGR2340": [False, 1],
        "ENGR2360": [False, 1],
        "ENGR3330": [False, 1],
        "ENGR3110/ENGR3180/ENGR3232/ENGR3299C/ENGR3260/ENGR3345/ENGR3350/ENGR3370/ENGR3392/ENGR3820": [
            False,
            1,
        ],
        "MTH3120/MTH3150/MTH3170": [False, 1],
    },
}
bio_requirements = []
design_depth_requirements = []
probstat_requirements = []
matsci_requirements = []

## Define all initializations
- `sem_courses`
- `credits_took`
- `courses_took`

In [11]:
sem_courses = {
    "freshman fall": ["QEA1", "Modsim", "DesNat", "AHS"],
    "freshman spring": ["QEA2", "ISIM", "P&M", ],
    "sophomore fall": ["PIE",],
    "sophomore spring": ["CD", ],
    "junior fall": [],
    "junior spring": [],
    "senior fall": ["Capstone"],
    "senior spring": ["Capstone",],
}
courses_took = [
    "ENGX2000",  # QEA
    "MTH1111/SCI1111",  # ModSim
    "ENGR1200",  # DesNat
    "AHSE0112",  # OCO
    "ENGX2005",  # QEA 2
    "ENGR1125",  # ISIM
    "AHSE1515",  # P&M
    "ENGR2110",  # PIE
    "ENGR2250",  # CD
]

In [12]:
def get_empty_schedules():
    """
    get empty schedules from sem_courses
    Return:
        dictionary of how many course are free(should be filled in) for each semester.
    """
    emtpy_sem_courses = {}
    for semester, courses in sem_courses.items():
        emtpy_sem_courses[semester] = courses.count("")
    return emtpy_sem_courses
get_empty_schedules()

{'freshman fall': 0,
 'freshman spring': 0,
 'sophomore fall': 0,
 'sophomore spring': 0,
 'junior fall': 0,
 'junior spring': 0,
 'senior fall': 0,
 'senior spring': 0}

## Get Course Type
separate all courses

In [13]:
def get_course_type(course):
    '''
    Determine the course type based on its prefix
    Args:
        course(str): Course code (e.g., 'MTH101', 'ENGR200')
    Returns:
        str: Course type ('MTH', 'ENG', 'SCI', 'AHS') or None if not matched
    '''
    if course.startswith('MTH'):
        return 'MTH'
    elif course.startswith('ENG'):
        return 'ENG'
    elif course.startswith('SCI'):
        return 'SCI'
    elif course.startswith('AHS'):
        return 'AHS'
    else:
        return None

## Get possible Courses
Get possible course for each semester as a dictionary along with its liklihood of offering


In [16]:
import pandas as pd

def get_possible_courses(semester):
    '''
    Get possible course for each semester as a dictionary along with its likelihood of offering
    Args:
        semester(str): semester that you want to check the course offering for (e.g., 'freshman fall')
        df (DataFrame): DataFrame containing the course offering data
    Returns:
        course offering dictionary with likelihood, separated by course type
    '''
    # Initialize an empty dictionary to store offered courses by type
    offered_courses = {'MTH': {}, 'ENG': {}, 'SCI': {}, 'AHS': {}}
    
    # Find the corresponding column index for the semester
    semester_column = semester.lower()
    
    # Check if the semester column exists in the DataFrame
    if semester_column in df.columns:
        # Filter the DataFrame for courses offered in the specified semester with values greater than 0.5
        courses_offered = df[df[semester_column] > 0.5]
        
        # Iterate through the courses offered and categorize them by type
        for course in courses_offered.index.tolist():
            course_type = get_course_type(course)
            if course_type:
                offered_courses[course_type][course] = courses_offered.loc[course, semester_column]
    
    return offered_courses

# Example usage
df = pd.read_csv("predicted_schedule.csv", index_col=0).T
semester = 'freshmen fall'
possible_courses = get_possible_courses(semester)
# print(f"Courses offered in {semester}:")
# for course_type, courses in possible_courses.items():
#     print(f"{course_type}:")
#     for course, likelihood in courses.items():
#         print(f"  {course}: {likelihood}")
possible_courses['AHS']


{}

## Get Possible Semesters

In [4]:
import pandas as pd
df = pd.read_csv('predicted_schedule.csv', index_col=0).T

def get_possible_semesters(course_number):
    '''
    Extracts semesters with a likelihood greater than 0.5 for a given course number ordered by likelihood.

    Args:
        df (DataFrame): DataFrame containing the likelihood of the course being offered for different semesters.
        course_number (str): The course number for which semesters are to be extracted.

    Returns:
        list: An ordered list of semesters with likelihood greater than 0.5 for the given course number.
    '''
    # Remove the first element (Course Title) from the row and make it into DataFrame
    possible_semesters_df = pd.DataFrame(df[course_number][1:])
    
    # Reset the index to have the semester as a column
    possible_semesters_df.reset_index(inplace=True)
    possible_semesters_df.columns = ['Semester', 'Likelihood']

    # Sort the DataFrame by Likelihood in ascending order
    possible_semesters_df = possible_semesters_df.sort_values(by='Likelihood', ascending=False)

    # Filter the DataFrame to select semesters with likelihood > 0.5
    possible_semesters = possible_semesters_df[possible_semesters_df['Likelihood'] > 0.5]['Semester'].tolist()

    return possible_semesters

# Example Usage
print(get_possible_semesters('ENGR1125'))

['freshmen spring', 'sophomore spring', 'junior spring', 'senior spring']


## Calculate Credits Taken

In [ ]:
def calculate_credits_taken(courses_took):
    '''
    Calculate the number of credits taken for each course type and total credits
    Args:
        courses_took(list): List of course codes taken
    Returns:
        dict: Dictionary containing the number of credits taken for each course type and total credits
    '''
    # Initialize credits
    credits_took = {
    "AHS": 0,
    "ENG": 0,
    "MTH/SCI": 0,
    "MTH": 0,
    "SCI": 0,
    "OFYI": 1,
    "TOTAL": 1,
}
    for course in courses_took:
        course_type = get_course_type(course)
        if course_type:
            # If the course type is valid, add the corresponding credits
            if course_type == 'MTH' or course_type == 'SCI':
                credits_took['MTH/SCI'] += 4
            credits_took[course_type] += 4
            credits_took['TOTAL'] += 4
    
    return credits_took

# Example Usage

courses_took = [
    "ENGX2000",  # QEA
    "MTH1111/SCI1111",  # ModSim
    "ENGR1200",  # DesNat
    "AHSE0112",  # OCO
    "ENGX2005",  # QEA 2
    "ENGR1125",  # ISIM
    "AHSE1515",  # P&M
    "ENGR2110",  # PIE
    "ENGR2250",  # CD
]

# Calculate credits taken
credits_took = calculate_credits_taken(courses_took)

# Print the credits taken
# print("Credits taken:")
# for course_type, credits in credits_took.items():
#     print(f"{course_type}: {credits}")
credits_took

{'AHS': 8, 'ENG': 24, 'MTH/SCI': 4, 'MTH': 4, 'SCI': 0, 'OFYI': 1, 'TOTAL': 37}

## fill_required_courses

check `fill_required_courses.py`

The resulting `sem_course` and `courses_took` will look like below.

In [ ]:
sem_courses = {
    "Freshmen Fall": ["QEA1", "Modsim", "DesNat", "AHS"],
    "Freshmen Spring": ["QEA2", "ISIM", "P&M"],
    "Sophomore Fall": [
        "PIE",
        "ENGX2010: Quantitative Engineering Analysis 3",
        "ENGX2134: Engineering Systems Analysis (2cr)",
        "ENGR2360: Introduction to Thermal-Fluid Systems",
    ],
    "Sophomore Spring": [
        "CD",
        "Mechanics of Solids and Structures",
        "ENGR2340: Dynamics",
        "MTH3120: Partial Differential Equations",
    ],
    "Junior Fall": ["Mechanical Design", "Design for Manufacturing"],
    "Junior Spring": [],
    "Senior Fall": ["Capstone"],
    "Senior Spring": ["Capstone"],
}
courses_took = [
    "ENGX2000",
    "MTH1111/SCI1111",
    "ENGR1200",
    "AHSE0112",
    "ENGX2005",
    "ENGR1125",
    "AHSE1515",
    "ENGR2110",
    "ENGR2250",
    "ENGR3525",
    "ENGR3599A",
    "ENGR3220",
    "ENGR3599",
]

## Fill Other requirements

In [ ]:
other_requirements = {
    "bio":['SCI1270','SCI1260/AHSE2160'],
    "design_depth" :['ENGR3210','ENGR3290','ENGR3250'],
    "probstat" : ['MTH2130', ],
    "matsci" :['SCI1440']
}
other_requirements_fulfilled = {'bio':False, 'design_depth':False, 'probstat':False, 'matsci':False}

In [ ]:
sem_courses['freshmen fall']

['QEA1', 'Modsim', 'DesNat', 'AHS']

In [ ]:
def fill_other_requirements(MAX_COURSES=4):
    for (
        course_type,
        courses,
    ) in other_requirements.items():  # courses is list of course number
        for course in courses:
            # among the semester the course would be offered,
            possible_semesters = get_possible_semesters(course)
            for semester in possible_semesters:
                # if the person is taking less courses than MAX_COURSES in the semester
                if (
                    len(sem_courses[semester]) < MAX_COURSES
                    and not other_requirements_fulfilled[course_type]
                ):
                    # fill in the sem_courses
                    course_title = df[course]["Course Title"]
                    sem_courses[semester].append(course_title)
                    # add it to the courses_took
                    courses_took.append(course)
                    other_requirements_fulfilled[course_type] = True
                    break
    return sem_courses

# Example Usage
fill_other_requirements()

{'freshman fall': ['QEA1', 'Modsim', 'DesNat', 'AHS'],
 'freshman spring': ['QEA2',
  'ISIM',
  'P&M',
  'SCI1270: Biomes, Climate Change, and Biodiversity (BCB)'],
 'sophomore fall': ['PIE',
  'SCI1270: Biomes, Climate Change, and Biodiversity (BCB)',
  'Affordable Design and Entrepreneurship'],
 'sophomore spring': ['CD',
  'SCI1260_AHSE2160: The Intersection of Biology, Art and Technology',
  'ENGR3210: Sustainable Design',
  'Affordable Design and Entrepreneurship'],
 'junior fall': [],
 'junior spring': ['Integrated Product Design',
  'Probability and Statistics',
  'SCI1270: Biomes, Climate Change, and Biodiversity (BCB)',
  'SCI1260_AHSE2160: The Intersection of Biology, Art and Technology'],
 'senior fall': ['Capstone',
  'SCI1440 Materials Creation, Consumption, and Impact'],
 'senior spring': ['Capstone',
  'ENGR3210: Sustainable Design',
  'Affordable Design and Entrepreneurship',
  'Integrated Product Design']}

After running the code sem_courses becomes as below

In [48]:
sem_courses = {
    "freshmen fall": ["QEA1", "Modsim", "DesNat", "AHS"],
    "freshmen spring": [
        "QEA2",
        "ISIM",
        "P&M",
        "SCI1270: Biomes, Climate Change, and Biodiversity (BCB)",
    ],
    "sophomore fall": [
        "PIE",
        "ENGX2010: Quantitative Engineering Analysis 3",
        "ENGX2134: Engineering Systems Analysis (2cr)",
        "ENGR2360: Introduction to Thermal-Fluid Systems",
    ],
    "sophomore spring": [
        "CD",
        "Mechanics of Solids and Structures",
        "ENGR2340: Dynamics",
        "MTH3120: Partial Differential Equations",
    ],
    "junior fall": ["Mechanical Design", "Design for Manufacturing"],
    "junior spring": [
        "ENGR3210: Sustainable Design",
        "Probability and Statistics",
        "SCI1440 Materials Creation, Consumption, and Impact",
    ],
    "senior fall": ["Capstone"],
    "senior spring": ["Capstone"],
}
courses_took = ['ENGX2000', 'MTH1111/SCI1111', 'ENGR1200', 'AHSE0112', 'ENGX2005', 'ENGR1125', 'AHSE1515', 'ENGR2110', 'ENGR2250', 'ENGR3525', 'ENGR3599A', 'ENGR3220', 'ENGR3599', 'SCI1270', 'ENGR3210', 'SCI1440']

In [ ]:
len(courses_took)

16

# fill_empty_schedules

In [ ]:
# credits_took = calculate_credits_taken(courses_took)

credits_took = {'AHS': 8,
 'ENG': 68,
 'MTH/SCI': 16,
 'MTH': 8,
 'SCI': 8,
 'OFYI': 1,
 'TOTAL': 93}
credits_needed = {}
for course_type, credits in credits_required.items():
    credits_needed[course_type] = credits - credits_took[course_type]

credits_needed

{'AHS': 20,
 'ENG': -22,
 'MTH/SCI': 14,
 'MTH': 2,
 'SCI': -8,
 'OFYI': 0,
 'TOTAL': 27}

In [ ]:
get_possible_courses('sophomore fall',df)['ENG']

{'ENGR1200': 0.9990832452217484,
 'ENGR1330': 0.8867005730802183,
 'ENGR2110': 0.8285211050533808,
 'ENGR2250': 1.0000001985092457,
 'ENGR2299': 0.6030968114016738,
 'ENGR2360': 0.5836093516847884,
 'ENGR3220': 0.8089937261587784,
 'ENGR3260': 0.8589347380702645,
 'ENGR3260L': 0.8899791614648302,
 'ENGR3290': 1.0,
 'ENGR3299A': 0.5992454364481712,
 'ENGR3330': 0.970105218660164,
 'ENGR3350': 0.7154293016895803,
 'ENGR3355': 1.0000000000003757,
 'ENGR3365': 1.0000000000003757,
 'ENGR3392': 1.1018793466643857,
 'ENGR3410': 0.9990832452217484,
 'ENGR3420': 0.7707658682892973,
 'ENGR3499': 0.5284556286647817,
 'ENGR3520': 1.0002990402477832,
 'ENGR3599': 1.000074917008313,
 'ENGR4190': 1.0,
 'ENGR4290': 1.0,
 'ENGR4599': 0.9126261341298476,
 'ENGX2000': 0.8382362394045956,
 'ENGX2010': 0.8731955754090999,
 'ENGX2134': 1.0000000000003757}

## choose course 
choose a course randomly for the given semester based on weighing course's likihood of offering 

In [47]:
import random
def choose_course(semester, course_type="ALL"):
     """
     choose a course for the course_type and semester according to its liklihood of offering
     if course_type is ALL, choose from any subject

     Args:
      course_type:
      semester:

     return:
          course num
     """
     if course_type == "ALL":
          possible_courses = get_possible_courses(semester, df)
          all_possible_courses = {}
          for courses in possible_courses.values():
               all_possible_courses.update(courses)

          possible_courses = all_possible_courses
          print(possible_courses)
     else: # if there is a choosen course type
          possible_courses = get_possible_courses(semester, df)[course_type]
     # Extract course names and their weights
     course_names = list(possible_courses.keys())
     weights = list(possible_courses.values())

     # Randomly choose a course based on weights
     random_course = random.choices(course_names, weights=weights)[0]
     return random_course


choose_course("freshmen spring", 'MTH')


'MTH3120'

## fill empty schedules
fill any empty schedules according to credits needed with the choosen courses from the choose_course function

In [64]:
df = pd.read_csv('predicted_schedule.csv', index_col=0)
"""
fill in the sem_course with the possible_courses
"""
MAX_NUM_COURSE = 4
def fill_empty_schedules(sem_courses):
    '''
    Fill courses in semesters based on major requirements, course offerings, and credit requirements.
    '''
    
    while credits_needed['TOTAL']>0:
        # fill MTH if not enough math credit 
        if credits_needed['MTH']>0:
            for semester, courses in sem_courses.items():
                if len(courses) < MAX_NUM_COURSE:
                    choosen_course = choose_course(semester, 'MTH') # course num
                    choosen_course_title = df.T[choosen_course]["Course Title"] # course title
                    sem_courses[semester].append(choosen_course_title)
                    courses_took.append(choosen_course)
                    credits_needed['MTH']-=4
                    credits_needed['MTH/SCI']-=4
                    credits_needed['TOTAL']-=4
        # fill MTH/SCI
        elif credits_needed['MTH/SCI']>0:
            for semester, courses in sem_courses.items():
                if len(courses) < MAX_NUM_COURSE:
                    # if MTH/SCI just fill in with SCI courses??
                    choosen_course = choose_course(semester, 'SCI') # course num 
                    choosen_course_title = df.T[choosen_course]["Course Title"] # course title
                    sem_courses[semester].append(choosen_course_title)
                    courses_took.append(choosen_course)
                    credits_needed['SCI']-=4
                    credits_needed['MTH/SCI']-=4
                    credits_needed['TOTAL']-=4
        # fill ENG
        elif credits_needed['ENG']>0:
            for semester, courses in sem_courses.items():
                if len(courses) < MAX_NUM_COURSE:
                    choosen_course = choose_course(semester, 'SCI') # course num
                    choosen_course_title = df.T[choosen_course]["Course Title"] # course title
                    sem_courses[semester].append(choosen_course_title)
                    courses_took.append(choosen_course)
                    credits_needed['ENG']-=4
                    credits_needed['TOTAL']-=4
        # fill AHS --> just 'AHS', no calculation
        elif credits_needed['AHS']>0:
            for semester, courses in sem_courses.items():
                if len(courses) < MAX_NUM_COURSE:
                    sem_courses[semester].append('AHS')
                    courses_took.append('AHS')
                    credits_needed['AHS']-=4
                    credits_needed['TOTAL']-=4
        # fill TOTAL with any course
        else:
            for semester, courses in sem_courses.items():
                if len(courses) < MAX_NUM_COURSE:
                    choosen_course = choose_course(semester) # course num
                    choosen_course_title = df.T[choosen_course]["Course Title"] # course title
                    sem_courses[semester].append(choosen_course_title)
                    courses_took.append(choosen_course)
                    credits_needed[choosen_course[0:3]]-=4 # course type
                    credits_needed['TOTAL']-=4
    return sem_courses

fill_empty_schedules(sem_courses)
            



{'freshmen fall': ['QEA1', 'Modsim', 'DesNat', 'AHS'],
 'freshmen spring': ['QEA2',
  'ISIM',
  'P&M',
  'SCI1270: Biomes, Climate Change, and Biodiversity (BCB)'],
 'sophomore fall': ['PIE',
  'ENGX2010: Quantitative Engineering Analysis 3',
  'ENGX2134: Engineering Systems Analysis (2cr)',
  'ENGR2360: Introduction to Thermal-Fluid Systems'],
 'sophomore spring': ['CD',
  'Mechanics of Solids and Structures',
  'ENGR2340: Dynamics',
  'MTH3120: Partial Differential Equations'],
 'junior fall': ['Mechanical Design',
  'Design for Manufacturing',
  'MTH3120: Partial Differential Equations',
  'AHS'],
 'junior spring': ['ENGR3210: Sustainable Design',
  'Probability and Statistics',
  'SCI1440 Materials Creation, Consumption, and Impact',
  'MTH2136 / SCI2136: Astronomy and Statistics: AstroStats'],
 'senior fall': ['Capstone',
  'Modeling and Simulation of the Physical World',
  'AHS'],
 'senior spring': ['Capstone',
  'MTH2136 / SCI2136: Astronomy and Statistics: AstroStats',
  'AHS']

## Finalized Course Schedule

In [65]:
sem_courses

{'freshmen fall': ['QEA1', 'Modsim', 'DesNat', 'AHS'],
 'freshmen spring': ['QEA2',
  'ISIM',
  'P&M',
  'SCI1270: Biomes, Climate Change, and Biodiversity (BCB)'],
 'sophomore fall': ['PIE',
  'ENGX2010: Quantitative Engineering Analysis 3',
  'ENGX2134: Engineering Systems Analysis (2cr)',
  'ENGR2360: Introduction to Thermal-Fluid Systems'],
 'sophomore spring': ['CD',
  'Mechanics of Solids and Structures',
  'ENGR2340: Dynamics',
  'MTH3120: Partial Differential Equations'],
 'junior fall': ['Mechanical Design',
  'Design for Manufacturing',
  'MTH3120: Partial Differential Equations',
  'AHS'],
 'junior spring': ['ENGR3210: Sustainable Design',
  'Probability and Statistics',
  'SCI1440 Materials Creation, Consumption, and Impact',
  'MTH2136 / SCI2136: Astronomy and Statistics: AstroStats'],
 'senior fall': ['Capstone',
  'Modeling and Simulation of the Physical World',
  'AHS'],
 'senior spring': ['Capstone',
  'MTH2136 / SCI2136: Astronomy and Statistics: AstroStats',
  'AHS']